In [71]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [72]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [73]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [74]:
metrics = loadMetricData('sallyKung', "metrics")
metrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-09 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN
1436,2016-05-09 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN
1437,2016-05-09 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN
1438,2016-05-09 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN
1439,2016-05-09 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
metrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [76]:
# clean metric data

metrics.timestamp = pd.to_datetime(metrics.timestamp)

metrics.heartrate = metrics.heartrate.fillna(0)
metrics.steps = metrics.steps.fillna(0)
metrics.calories = metrics.calories.fillna(0)
metrics.gsr = metrics.gsr.fillna(0)
metrics.skintemp = metrics.skintemp.fillna(0)
metrics.airtemp = metrics.airtemp.fillna(0)

# metrics.head(5)

In [77]:
metrics = metrics[metrics.heartrate != 0]
metrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
842,2016-05-02 14:02:00,93,4,1.709,0.000000,0.0,0.0
843,2016-05-02 14:03:00,99,18,2.000,0.000052,84.2,82.4
844,2016-05-02 14:04:00,91,12,1.800,0.000054,84.2,82.4
845,2016-05-02 14:05:00,93,0,1.500,0.000054,84.2,82.4
846,2016-05-02 14:06:00,80,0,1.100,0.000053,84.2,82.4


In [78]:
activities = loadMetricData('sallyKung', "activities")
# activities.tail(5)

In [79]:
sleep = loadMetricData('sallyKung', "sleep")
# sleep.tail(5)

In [80]:
# sleep.dtypes

In [81]:
journal = loadMetricData('sallyKung', "journal")
# journal.tail(5)

In [82]:
# journal.dtypes

In [83]:
# clean metric data
journal.starttime= pd.to_datetime(journal.starttime)
journal.endtime= pd.to_datetime(journal.endtime)

journal.qualitative = journal.qualitative.fillna(0)
journal.excited = journal.excited.fillna(0)
journal.happy = journal.happy.fillna(0)
journal.calm = journal.calm.fillna(0)
journal.anxious = journal.anxious.fillna(0)
journal.sad = journal.sad.fillna(0)
journal.angry = journal.angry.fillna(0)
journal.hungry = journal.hungry.fillna(0)
journal.tired = journal.tired.fillna(0)
journal.bored = journal.bored.fillna(0)
journal.ansaan = journal.ansaan.fillna(0)
journal.exhaca = journal.exhaca.fillna(0)

print(journal.shape)
journal.head(5)
# journal.tail(5)

(46, 24)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
0,2016-05-02 13:25:00,2016-05-02 13:28:00,5/2/16,1:25 PM,1:28 PM,Trying on watch,Barb's Office,Excited,8,8,...,0,0,0,0,16,0,16,8,0,0
1,2016-05-02 13:40:00,2016-05-02 00:00:00,5/2/16,1:40 PM,NaN,Editing photos for work,Barb's Office,Streeeeeessed,6,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-05-02 13:48:00,2016-05-02 13:48:00,5/2/16,1:48 PM,1:48 PM,Wafer fit perfectly on top of cup,Barb's Office,HAPPY,10,10,...,0,0,0,0,20,0,20,10,0,0
3,2016-05-02 14:31:00,2016-05-02 14:50:00,5/2/16,2:31 PM,2:50 PM,Sorting through magnetic pins,Barb's Office,Anxious,5,0,...,6,0,0,0,0,11,0,0,11,5
4,2016-05-02 15:10:00,2016-05-02 15:14:00,5/2/16,3:10 PM,3:14 PM,Trying to figure out where photos went,Barb's Office,"ANXIOUS, CONFUSED",9,0,...,0,0,0,0,0,9,0,0,9,9


In [84]:

journal['boolexcited'] =  journal['excited'] > 0
journal['boolhappy'] =  journal['happy'] > 0
journal['boolcalm'] =  journal['calm'] > 0
journal['boolanxious'] =  journal['anxious'] > 0
journal['boolsad'] =  journal['sad'] > 0
journal['boolangry'] =  journal['angry'] > 0
journal['booltired'] =  journal['tired'] > 0
journal['boolhungry'] =  journal['hungry'] > 0
journal['boolbored'] =  journal['bored'] > 0
journal['exhaca'] =  journal['exhaca'] > 0
journal['ansaan'] =  journal['ansaan'] > 0

journal['exha'] =  journal['exha'] > 0
journal['haca'] =  journal['haca'] > 0
journal['anan'] =  journal['anan'] > 0
journal['ansa'] =  journal['anan'] > 0

    
boolexcited = journal.iloc[:, 18].values.reshape(journal.shape[0], 1)
boolhappy = journal.iloc[:, 19].values.reshape(journal.shape[0], 1)
boolcalm = journal.iloc[:, 20].values.reshape(journal.shape[0], 1)
boolanxious = journal.iloc[:, 21].values.reshape(journal.shape[0], 1)
boolsad = journal.iloc[:, 22].values.reshape(journal.shape[0], 1)
boolangry = journal.iloc[:, 23].values.reshape(journal.shape[0], 1)
booltired = journal.iloc[:, 24].values.reshape(journal.shape[0], 1)
boolhungry = journal.iloc[:, 25].values.reshape(journal.shape[0], 1)
boolbored = journal.iloc[:, 26].values.reshape(journal.shape[0], 1)
exhaca = journal.iloc[:, 27].values.reshape(journal.shape[0], 1)
ansaan = journal.iloc[:, 28].values.reshape(journal.shape[0], 1)
exha = journal.iloc[:, 29].values.reshape(journal.shape[0], 1)
haca = journal.iloc[:, 30].values.reshape(journal.shape[0], 1)
anan = journal.iloc[:, 31].values.reshape(journal.shape[0], 1)
ansa = journal.iloc[:, 32].values.reshape(journal.shape[0], 1)

journal.head(5)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,ansa,boolexcited,boolhappy,boolcalm,boolanxious,boolsad,boolangry,booltired,boolhungry,boolbored
0,2016-05-02 13:25:00,2016-05-02 13:28:00,5/2/16,1:25 PM,1:28 PM,Trying on watch,Barb's Office,Excited,8,8,...,False,True,True,False,False,False,False,False,False,False
1,2016-05-02 13:40:00,2016-05-02 00:00:00,5/2/16,1:40 PM,NaN,Editing photos for work,Barb's Office,Streeeeeessed,6,0,...,False,False,False,False,False,False,False,False,False,False
2,2016-05-02 13:48:00,2016-05-02 13:48:00,5/2/16,1:48 PM,1:48 PM,Wafer fit perfectly on top of cup,Barb's Office,HAPPY,10,10,...,False,True,True,False,False,False,False,False,False,False
3,2016-05-02 14:31:00,2016-05-02 14:50:00,5/2/16,2:31 PM,2:50 PM,Sorting through magnetic pins,Barb's Office,Anxious,5,0,...,True,False,False,False,True,False,True,False,False,False
4,2016-05-02 15:10:00,2016-05-02 15:14:00,5/2/16,3:10 PM,3:14 PM,Trying to figure out where photos went,Barb's Office,"ANXIOUS, CONFUSED",9,0,...,True,False,False,False,True,False,False,False,False,False


In [85]:
# for i in range(0, sarahJournal.shape[0]):
#     sarahJournal['ex'] = (0< sarahJournal.iloc[i, 18].astype(int))
#     sarahJournal['ha'] = sarahJournal.iloc[i, 19]== True
#     sarahJournal['ca'] = sarahJournal.iloc[i, 20]== True
#     sarahJournal['ys'] = sarahJournal['ex'].astype(int) + sarahJournal['ha'].astype(int) + sarahJournal['ca'].astype(int)

# print(sarahJournal['ex'])
    

In [86]:
# def makeEmoIndicators(newvar, journalData, journal_emo1_loc, journal_emo2_loc, journal_emo3_loc):
  
#     journalData[newvar] = False
    
#     for i in range(0, journalData.shape[0]):
#         journalData['emo1'] = (journalData.iloc[i, journal_emo1_loc]) == True)
#         journalData['emo2'] = (journalData.iloc[i, journal_emo2_loc] == True)
#         journalData['emo3'] = (journalData.iloc[i, journal_emo3_loc] == True)
#         journalData['ysum'] = journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int)
# #         print(journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int))
#         journalData.loc[journalData['ysum']!=0, newvar] = True
        
#     print(journalData['emo1'].values)
#     journalData = journalData.drop(['emo1', 'emo2', 'emo3', 'ysum'], axis=1)
#     return journalData

In [87]:
# sarahJournal.dtypes

In [88]:
# testData= makeEmoIndicators('exhaca', sarahJournal, 18, 19, 20)
# # print(testData.describe())

In [89]:
# sarahJournal.boolexhapcal.convert_objects(convert_numeric=True)

In [90]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [91]:
# indicator for walking
totalData = makeIndicators('excitement', metrics, journal, 0, 1, 18)
totalData = makeIndicators('happy', metrics, journal, 0, 1, 19)
totalData = makeIndicators('calm', metrics, journal, 0, 1, 20)
totalData = makeIndicators('anxious', metrics, journal, 0, 1, 21)
totalData = makeIndicators('sad', metrics, journal, 0, 1, 22)
totalData = makeIndicators('angry', metrics, journal, 0, 1, 23)
totalData = makeIndicators('hungry', metrics, journal, 0, 1, 24)
totalData = makeIndicators('tired', metrics, journal, 0, 1, 25)
totalData = makeIndicators('bored', metrics, journal, 0, 1, 26)

totalData = makeIndicators('exhaca', metrics, journal, 0, 1, 27)
totalData = makeIndicators('ansaan', metrics, journal, 0, 1, 28)

totalData = makeIndicators('exha', metrics, journal, 0, 1, 29)
totalData = makeIndicators('haca', metrics, journal, 0, 1, 30)
totalData = makeIndicators('anan', metrics, journal, 0, 1, 31)
totalData = makeIndicators('ansa', metrics, journal, 0, 1, 32)




In [92]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  10055.000000  10055.000000  10055.000000  10055.000000  10055.000000   
mean      79.915664      4.050622      1.820627      0.001328     85.704267   
std       16.522298     18.208038      1.359876      0.050552      9.483957   
min       51.000000      0.000000      0.700000      0.000000      0.000000   
25%       66.500000      0.000000      1.200000      0.000053     83.300000   
50%       79.000000      0.000000      1.300000      0.000055     86.000000   
75%       90.000000      0.000000      1.900000      0.000062     89.600000   
max      156.000000    142.000000     11.400000      4.270000     99.500000   

            airtemp excitement     happy       calm    anxious     ...      \
count  10055.000000      10055     10055      10055      10055     ...       
mean      82.895425  0.0277474  0.082546  0.0016907  0.0277474     ...       
std        9.461043   0.164256  0.275208  0.0410854   

In [93]:
print(totalData.head())

              timestamp  heartrate  steps  calories       gsr  skintemp  \
842 2016-05-02 14:02:00         93      4     1.709  0.000000       0.0   
843 2016-05-02 14:03:00         99     18     2.000  0.000052      84.2   
844 2016-05-02 14:04:00         91     12     1.800  0.000054      84.2   
845 2016-05-02 14:05:00         93      0     1.500  0.000054      84.2   
846 2016-05-02 14:06:00         80      0     1.100  0.000053      84.2   

     airtemp excitement  happy   calm  ...    angry hungry  tired  bored  \
842      0.0      False  False  False  ...    False  False  False  False   
843     82.4      False  False  False  ...    False  False  False  False   
844     82.4      False  False  False  ...    False  False  False  False   
845     82.4      False  False  False  ...    False  False  False  False   
846     82.4      False  False  False  ...    False  False  False  False   

    exhaca ansaan   exha   haca   anan   ansa  
842  False  False  False  False  False  Fals

In [94]:
totalData.to_csv('sarahTotal.csv')

In [95]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
842,2016-05-02 14:02:00,93,4,1.709,0.000000,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
843,2016-05-02 14:03:00,99,18,2.000,0.000052,84.2,82.4,False,False,False,...,False,False,False,False,False,False,False,False,False,False
844,2016-05-02 14:04:00,91,12,1.800,0.000054,84.2,82.4,False,False,False,...,False,False,False,False,False,False,False,False,False,False
845,2016-05-02 14:05:00,93,0,1.500,0.000054,84.2,82.4,False,False,False,...,False,False,False,False,False,False,False,False,False,False
846,2016-05-02 14:06:00,80,0,1.100,0.000053,84.2,82.4,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [96]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(10055, 26)
              timestamp  heartrate  steps  calories       gsr  skintemp  \
842 2016-05-02 14:02:00         93      4     1.709  0.000000       0.0   
843 2016-05-02 14:03:00         99     18     2.000  0.000052      84.2   
844 2016-05-02 14:04:00         91     12     1.800  0.000054      84.2   
845 2016-05-02 14:05:00         93      0     1.500  0.000054      84.2   
846 2016-05-02 14:06:00         80      0     1.100  0.000053      84.2   

     airtemp excitement  happy   calm ... exhaca ansaan   exha   haca   anan  \
842      0.0      False  False  False ...  False  False  False  False  False   
843     82.4      False  False  False ...  False  False  False  False  False   
844     82.4      False  False  False ...  False  False  False  False  False   
845     82.4      False  False  False ...  False  False  False  False  False   
846     82.4      False  False  False ...  False  False  False  False  False   

      ansa weekday month hour min  
842  False       0  

In [97]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
# exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
# thesis = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
# sally = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
# barb = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
# sol = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
# arash = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
# work = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
# home = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 7].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 8].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 9].values.reshape(totalData.shape[0], 1)

In [98]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
#         exercise, 
#         thesis, 
#         sally, 
#         barb, 
#         sol, 
#         arash, 
#         work, 
#         home, 
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(10055, 9)

# Positive ( Excited, Happy, Calm)

In [99]:
# define y


y = totalData.exhaca.values

# y = totalData.excitement.values
y.shape

(10055,)

In [100]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [101]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [102]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

## Positive ( Excited, Happy, Calm) Model Results

In [103]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00312588803637 0.00165727543918
 Accuracy Rate   0.996874111964   0.998342724561
True Positives              1.0              1.0
True Negatives               --   0.998185117967


(0.0016572754391779914, 0.99834272456082196, 1.0, 0.99818511796733211)

In [104]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00312588803637 0.00165727543918
      Accuracy Rate   0.996874111964   0.998342724561
     True Positives              1.0              1.0
     True Negatives               --   0.998185117967


(0.0016572754391779914, 0.99834272456082196, 1.0, 0.99818511796733211)

In [105]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00326797385621 0.00165727543918
 Accuracy Rate   0.996732026144   0.998342724561
True Positives              1.0              1.0
True Negatives               --   0.998185117967


(0.0016572754391779914, 0.99834272456082196, 1.0, 0.99818511796733211)

In [106]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00312588803637 0.00165727543918
 Accuracy Rate   0.996874111964   0.998342724561
True Positives              1.0              1.0
True Negatives               --   0.998185117967


(0.0016572754391779914, 0.99834272456082196, 1.0, 0.99818511796733211)

# Negative ( Anxious, Sad, Angry)

In [107]:
# define y
y = totalData.ansaan.values
y.shape

(10055,)

In [108]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [109]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Negative ( Anxious, Sad, Angry) Model Results

In [110]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000142085819835      0.0
 Accuracy Rate     0.99985791418      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [111]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000142085819835      0.0
      Accuracy Rate     0.99985791418      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [112]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00440466041489 0.00397746105403
 Accuracy Rate   0.995595339585   0.996022538946
True Positives              0.0              nan
True Negatives               --   0.996022538946


(0.0039774610540271793, 0.99602253894597281, nan, 0.99602253894597281)

In [113]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0791418016482 0.0868412330129
 Accuracy Rate  0.920858198352  0.913158766987
True Positives             1.0             nan
True Negatives              --  0.913158766987


(0.086841233012926747, 0.91315876698707321, nan, 0.91315876698707321)

# Happy/Calm 

In [114]:
# define y
y = totalData.haca.values
y.shape

(10055,)

In [115]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [116]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Happy/Calm  Model Results

In [117]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0390736004547 0.0417633410673
 Accuracy Rate  0.960926399545  0.958236658933
True Positives  0.718336483932  0.744588744589
True Negatives              --  0.975951184494


(0.041763341067285381,
 0.95823665893271459,
 0.74458874458874458,
 0.97595118449389806)

In [118]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0439045183291 0.0467351673848
      Accuracy Rate  0.956095481671  0.953264832615
     True Positives  0.901701323251  0.878787878788
     True Negatives              --   0.95944005743


(0.046735167384819358,
 0.95326483261518069,
 0.87878787878787878,
 0.95944005743000715)

In [119]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0439045183291 0.0507126284388
 Accuracy Rate  0.956095481671  0.949287371561
True Positives  0.913043478261  0.883116883117
True Negatives              --  0.954773869347


(0.050712628438846538,
 0.9492873715611535,
 0.88311688311688308,
 0.95477386934673369)

In [120]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate  0.0747371412333 0.0772290354657
 Accuracy Rate   0.925262858767  0.922770964534
True Positives 0.00567107750473             0.0
True Negatives               --   0.99928212491


(0.077229035465694401, 0.92277096453430563, 0.0, 0.99928212491026558)

# Anxious/Angry

In [121]:
# define y
y = totalData.anan.values
y.shape

(10055,)

In [122]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [123]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Angry Model Results

In [124]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000142085819835      0.0
 Accuracy Rate     0.99985791418      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [125]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000142085819835      0.0
      Accuracy Rate     0.99985791418      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [126]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00440466041489 0.00397746105403
 Accuracy Rate   0.995595339585   0.996022538946
True Positives              0.0              nan
True Negatives               --   0.996022538946


(0.0039774610540271793, 0.99602253894597281, nan, 0.99602253894597281)

In [127]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000142085819835      0.0
 Accuracy Rate     0.99985791418      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

# Excited/Happy

In [128]:
# define y
y = totalData.exha.values
y.shape

(10055,)

In [129]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [130]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Excited/Happy Model Values

In [131]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00497300369423 0.00629764666888
 Accuracy Rate   0.995026996306   0.993702353331
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0062976466688763671, 0.99370235333112367, 0.0, 1.0)

In [132]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00497300369423 0.00629764666888
      Accuracy Rate   0.995026996306   0.993702353331
     True Positives              0.0              0.0
     True Negatives               --              1.0


(0.0062976466688763671, 0.99370235333112367, 0.0, 1.0)

In [133]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0706166524581 0.0759032151144
 Accuracy Rate  0.929383347542  0.924096784886
True Positives  0.971428571429  0.947368421053
True Negatives              --  0.923949299533


(0.07590321511435201,
 0.92409678488564795,
 0.94736842105263153,
 0.92394929953302196)

In [134]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00596760443308 0.00795492210805
 Accuracy Rate   0.994032395567   0.992045077892
True Positives              0.0              0.0
True Negatives               --   0.998332221481


(0.0079549221080543586, 0.99204507789194563, 0.0, 0.99833222148098733)

# Anxious/Sad

In [135]:
# define y
y = totalData.ansa.values
y.shape

(10055,)

In [136]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [137]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Sad Model Results

In [138]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000142085819835      0.0
 Accuracy Rate     0.99985791418      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [139]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000142085819835      0.0
      Accuracy Rate     0.99985791418      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [140]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00440466041489 0.00397746105403
 Accuracy Rate   0.995595339585   0.996022538946
True Positives              0.0              nan
True Negatives               --   0.996022538946


(0.0039774610540271793, 0.99602253894597281, nan, 0.99602253894597281)

In [141]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000142085819835      0.0
 Accuracy Rate     0.99985791418      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)